In [30]:
import numpy as np
from tempfile import TemporaryFile
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [128]:
features = 'potential,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes'
features_list = features.split(',')
features_list = features_list * 4
for idx, feature in enumerate(features_list):
    if idx/36 == 0:
        features_list[idx] = 'goal_keeper_' + features_list[idx]
    elif idx/36 == 1:
        features_list[idx] = 'defenders_' + features_list[idx]
    elif idx/36 == 2:
        features_list[idx] = 'midfielders_' + features_list[idx]
    else:
        features_list[idx] = 'strikers_' + features_list[idx]

In [119]:
def further_pruning(all_data,features_list):
    all_data = np.concatenate([all_data[:,31:67],all_data[:,72:103],all_data[:,108:139],all_data[:,-1].reshape([all_data.shape[0],1])],axis=1)
#                                ,all_data[:,175:211],all_data[:,216:247],all_data[:,252:283]
    features_list =  features_list[31:67] + features_list[72:103]+features_list[108:139]
#     +features_list[175:211]+features_list[216:247]+ features_list[252:283]
#     print all_data.shape
    return features_list,all_data
#     print all_data.shape


In [130]:
outfile = 'alldata.npz'
npzfile = np.load(outfile)
all_data = npzfile['arr_0']

(15965L, 289L)


In [131]:
print len(features_list)
features_list,all_data=further_pruning(all_data,features_list)


144


In [132]:
print all_data.shape
print len(features_list)

(15965L, 197L)
98


In [133]:
train_X,test_X,train_y,test_y = train_test_split(all_data[:,:-1],all_data[:,-1], test_size = 0.2, random_state = 4990)

In [134]:
C = np.linspace(0.01,10,10)
train_X  = (train_X-np.mean(train_X,axis = 0))/np.std(train_X,axis = 0)
test_X = (test_X - np.mean(train_X,axis = 0))/np.std(train_X,axis=0)
test_scores = np.zeros(10)
scores = np.zeros(10)
for (i,c) in enumerate(C):
    log = LogisticRegression(C = c,penalty='l1')
    cross = cross_val_score(log,train_X,train_y,cv = 5)
    scores[i] = np.mean(cross)
    log2 = LogisticRegression(C = c,penalty = "l1")
    log2.fit(train_X,train_y)
    test_scores[i] = log2.score(test_X,test_y)
    print c, scores[i],test_scores[i]

0.01 0.714530907352 0.672721578453
1.12 0.712025402002 0.71030378954
2.23 0.711712229157 0.710930159724
3.34 0.711399086962 0.71030378954
4.45 0.711320778427 0.71030378954
5.56 0.711320778427 0.709677419355
6.67 0.711242469891 0.709677419355
7.78 0.711320747778 0.708737864078
8.89 0.711242439242 0.708424678985
10.0 0.711242439242 0.708737864078


In [89]:
bestc = C[np.argmax(scores)]
log = LogisticRegression(C = bestc,penalty = "l1")
log.fit(train_X,train_y)

LogisticRegression(C=5.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [114]:
log.score(test_X,test_y)

0.63639210773567179

In [113]:
best_features = np.argsort(log.coef_).reshape(98)[::-1].reshape(98)
print best_features.shape
print len(features_list)
for idx in  best_features:
    print features_list[idx],log.coef_.reshape(98)[idx]


(98L,)
98
defenders_defensive_work_rate 0.280913522197
midfielders_defensive_work_rate 0.0714807334351
defenders_attacking_work_rate 0.0472774846181
strikers_attacking_work_rate 0.0397264579052
defenders_interceptions 0.0261350114489
midfielders_vision 0.0219913124048
strikers_vision 0.01919381832
midfielders_positioning 0.0184029771581
defenders_dribbling 0.0181031385376
defenders_agility 0.0177100811854
defenders_heading_accuracy 0.0171163867111
midfielders_marking 0.0171160179479
defenders_vision 0.016616799632
strikers_finishing 0.0152020837192
defenders_marking 0.0142633598817
goal_keeper_gk_kicking 0.0121940798791
midfielders_penalties 0.0114686397322
strikers_standing_tackle 0.0109278364699
midfielders_interceptions 0.0104718755357
midfielders_jumping 0.00925529006378
midfielders_volleys 0.00860363859849
defenders_standing_tackle 0.00857534141189
defenders_long_passing 0.00778329648563
defenders_strength 0.00747519409058
defenders_long_shots 0.00743950048044
strikers_jumping 0.0